<a href="https://colab.research.google.com/github/jdsonline/dupeguru/blob/master/Copy_of_Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Detective Adventure Game</title>
    <script src="https://cdn.tailwindcss.com"></script>
    <link href="https://fonts.googleapis.com/css2?family=Special+Elite&family=Roboto+Mono&display=swap" rel="stylesheet">
    <style>
        body {
            font-family: 'Roboto Mono', monospace;
            background-color: #1a1a1a;
            color: #e0e0e0;
        }
        .font-special-elite {
            font-family: 'Special Elite', cursive;
        }
        #story-container {
            border: 1px solid #444;
            background-color: #2b2b2b;
            box-shadow: 0 0 15px rgba(0,0,0,0.5);
            max-height: 55vh;
            overflow-y: auto;
        }
        .story-paragraph {
            border-bottom: 1px dashed #444;
            padding-bottom: 1rem;
            margin-bottom: 1rem;
        }
        .story-paragraph:last-child {
            border-bottom: none;
        }
        #input-form input {
            background-color: #333;
            border: 1px solid #555;
            color: #e0e0e0;
        }
        #input-form input:focus {
            outline: none;
            border-color: #777;
            box-shadow: 0 0 5px #777;
        }
        .btn {
            background-color: #4a4a4a;
            border: 1px solid #666;
            transition: all 0.3s ease;
        }
        .btn:hover {
            background-color: #5a5a5a;
            border-color: #888;
        }
        .btn-special {
             background-color: #3a5a5a;
             border-color: #668888;
        }
        .btn-special:hover {
            background-color: #5a7a7a;
             border-color: #88aaaa;
        }
        .btn-sm {
            padding: 0.25rem 0.5rem;
            font-size: 0.75rem;
        }
        .loader {
            border: 4px solid #f3f3f3;
            border-top: 4px solid #5a5a5a;
            border-radius: 50%;
            width: 40px;
            height: 40px;
            animation: spin 1s linear infinite;
        }
        @keyframes spin {
            0% { transform: rotate(0deg); }
            100% { transform: rotate(360deg); }
        }
        ::-webkit-scrollbar {
            width: 8px;
        }
        ::-webkit-scrollbar-track {
            background: #2b2b2b;
        }
        ::-webkit-scrollbar-thumb {
            background: #555;
            border-radius: 4px;
        }
        ::-webkit-scrollbar-thumb:hover {
            background: #777;
        }
    </style>
</head>
<body class="p-4 md:p-8">

    <div class="max-w-5xl mx-auto">
        <h1 class="text-4xl md:text-5xl font-special-elite text-center mb-4 text-red-300">Detective Virtual Game</h1>
        <p class="text-center mb-6 text-gray-400">The Case of the Crimson Quill</p>

        <div class="grid grid-cols-1 md:grid-cols-3 gap-6">
            <div class="md:col-span-2">
                 <div id="game-container" class="bg-gray-800/50 p-4 sm:p-6 rounded-lg shadow-2xl h-full flex flex-col">
                    <div id="story-container" class="flex-grow p-4 rounded-lg mb-4">
                        <!-- Story paragraphs will be injected here -->
                    </div>

                    <div id="loader" class="hidden justify-center items-center my-4">
                        <div class="loader"></div>
                        <p id="loader-text" class="ml-4">Thinking...</p>
                    </div>

                    <div id="input-area">
                        <form id="input-form" class="flex flex-col sm:flex-row gap-2">
                            <input type="text" id="player-input" class="flex-grow p-3 rounded-md focus:ring-2 focus:ring-red-400/50" placeholder="What do you do next? (e.g., 'examine the desk')">
                            <button type="submit" class="btn text-white font-bold py-3 px-6 rounded-md">Send</button>
                        </form>
                         <div id="quick-actions" class="mt-4 flex flex-wrap gap-2 justify-center">
                            <!-- Quick actions will be injected here -->
                        </div>
                    </div>
                </div>
            </div>
            <div class="flex flex-col gap-6">
                <div id="characters" class="p-4 bg-gray-800/50 rounded-lg shadow-xl">
                     <h2 class="text-xl font-special-elite text-red-200 mb-2">People in Room</h2>
                     <ul id="character-list" class="text-gray-300 space-y-2">
                        <!-- Characters will be listed here -->
                     </ul>
                </div>
                <div id="inventory" class="p-4 bg-gray-800/50 rounded-lg shadow-xl">
                     <h2 class="text-xl font-special-elite text-red-200 mb-2">Inventory</h2>
                     <ul id="inventory-list" class="space-y-2 text-gray-300">
                        <!-- Inventory items will be listed here -->
                     </ul>
                </div>
                 <div id="clues" class="p-4 bg-gray-800/50 rounded-lg shadow-xl flex flex-col">
                     <h2 class="text-xl font-special-elite text-red-200 mb-2">Known Clues</h2>
                     <ul id="clue-list" class="flex-grow list-disc list-inside text-gray-300">
                        <!-- Clues will be listed here -->
                     </ul>
                     <button id="reconstruct-btn" class="btn btn-special text-white p-2 rounded-md mt-4">✨ Reconstruct Scene</button>
                </div>
            </div>
        </div>
    </div>

    <script>
        // --- DOM Elements ---
        const storyContainer = document.getElementById('story-container');
        const inputForm = document.getElementById('input-form');
        const playerInput = document.getElementById('player-input');
        const loader = document.getElementById('loader');
        const loaderText = document.getElementById('loader-text');
        const inventoryList = document.getElementById('inventory-list');
        const characterList = document.getElementById('character-list');
        const clueList = document.getElementById('clue-list');
        const quickActionsContainer = document.getElementById('quick-actions');
        const reconstructButton = document.getElementById('reconstruct-btn');

        // --- Game State ---
        let gameState = {
            inventory: ["A trusty, but old, magnifying glass."],
            history: [],
            characters: {
                "Jeeves the Butler": { status: "standing by the door, looking pale and nervous", dialogue: [] },
                "Eleanor Blackwood": { status: "the victim's wife, sits in a high-backed chair, face a mask of cold composure", dialogue: [] },
                "Julian Blackwood": { status: "the victim's nephew, paces by the window, impatiently checking his watch", dialogue: [] }
            },
            known_clues: [],
            lastSpokenTo: null,
        };

        // --- Initial Scene ---
        const initialScene = {
            prompt: "You are a seasoned detective. You've been called to the private library of a reclusive millionaire, Alistair Blackwood. He's been found dead. The room is lavish, filled with old books, antique furniture, and the faint smell of rain and old paper. A single, ornate crimson quill lies on the floor near the body. The police have given you the run of the place. You are not alone. The family's long-serving butler, Jeeves, stands grim-faced by the doorway. Seated in a high-backed chair is the victim's wife, Eleanor, watching everything with an unnerving calm. By the window, Alistair's nephew, Julian, paces impatiently. The air is thick with tension and unspoken secrets. What do you do?",
        };

        // --- Game Logic ---
        async function startGame() {
            addStoryParagraph(initialScene.prompt, "narrator");
            const startPrompt = `Let's start the game. Here is the opening scenario: ${initialScene.prompt} The player's inventory is: ${gameState.inventory.join(', ')}. The characters present are: ${Object.keys(gameState.characters).join(', ')}.`;
            gameState.history.push({ role: "user", parts: [{ text: startPrompt }] });
            gameState.history.push({ role: "model", parts: [{ text: "I understand. I am ready to be the game master. The scene is set. I will await the player's first action." }] });
            updateUI();
        }

        function addStoryParagraph(text, speaker = "narrator") {
            const p = document.createElement('p');
            p.className = 'story-paragraph';
            p.innerHTML = text.replace(/\n/g, '<br>');

            if (speaker === 'player') { p.classList.add('text-red-300', 'italic'); p.textContent = `> ${text}`; }
            else if (speaker === 'hint') { p.classList.add('text-yellow-300', 'italic'); p.innerHTML = `(A thought strikes you...)<br>${text.replace(/\n/g, '<br>')}`; }
            else if (speaker === 'reconstruction') { p.classList.add('text-cyan-300', 'italic'); p.innerHTML = `<strong>[SCENE RECONSTRUCTION]</strong><br>${text.replace(/\n/g, '<br>')}`; }
            else if (speaker === 'analysis') { p.classList.add('text-purple-300', 'italic'); p.innerHTML = `<strong>[FORENSIC ANALYSIS]</strong><br>${text.replace(/\n/g, '<br>')}`; }

            storyContainer.appendChild(p);
            storyContainer.scrollTop = storyContainer.scrollHeight;
        }

        function updateUI() {
            // Inventory
            inventoryList.innerHTML = gameState.inventory.map(item => `
                <li class="flex justify-between items-center">
                    <span>- ${item}</span>
                    <button class="btn btn-sm btn-special analyze-btn" data-item="${item}">✨ Analyze</button>
                </li>`).join('') || '<li>Your pockets are empty.</li>';

            // Characters
            characterList.innerHTML = Object.entries(gameState.characters).map(([name, char]) => `<li><strong>${name}:</strong> ${char.status}</li>`).join('');

            // Quick Actions
            let quickActionsHTML = '<button class="btn quick-action-btn text-white p-2 rounded-md" data-action="look around the room">Look Around</button>';
            quickActionsHTML += Object.keys(gameState.characters).map(name => `<button class="btn quick-action-btn text-white p-2 rounded-md" data-action="talk to ${name}">Talk to ${name}</button>`).join('');
            if (gameState.lastSpokenTo && gameState.characters[gameState.lastSpokenTo]) {
                 quickActionsHTML += `<button class="btn quick-action-btn btn-special text-white p-2 rounded-md" data-action="Press ${gameState.lastSpokenTo} for details">✨ Press ${gameState.lastSpokenTo}</button>`;
            }
            quickActionsHTML += `<button class="btn quick-action-btn text-white p-2 rounded-md" data-action="I need a hint">Get a Hint</button>`;
            quickActionsContainer.innerHTML = quickActionsHTML;
            addEventListeners();

            // Clues
            clueList.innerHTML = gameState.known_clues.map(clue => `<li class="ml-4">${clue}</li>`).join('') || '<li class="ml-4">No solid clues yet.</li>';
        }

        async function handlePlayerAction(action) {
            if (!action.trim()) return;

            const actionType = getActionType(action);

            if (actionType.speaker === 'player') {
                addStoryParagraph(action, "player");
            }

            playerInput.value = '';
            showLoader(true, actionType.loaderText);

            const prompt = createPrompt(action);

            try {
                let chatHistory = actionType.useFullHistory ? [...gameState.history] : [];
                chatHistory.push({ role: "user", parts: [{ text: prompt }] });

                const payload = { contents: chatHistory };
                const apiKey = "" // IMPORTANT: A Google AI API key is required for this to function.
                const apiUrl = `https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent?key=${apiKey}`;

                const response = await fetch(apiUrl, { method: 'POST', headers: { 'Content-Type': 'application/json' }, body: JSON.stringify(payload) });
                if (!response.ok) {
                     if (response.status === 400) {
                         throw new Error(`API request failed: Bad request. Please ensure you have a valid Gemini API key.`);
                     }
                     throw new Error(`API request failed with status ${response.status}`);
                }

                const result = await response.json();
                const responseText = result.candidates?.[0]?.content?.parts?.[0]?.text;

                if (responseText) {
                    parseAndApplyResponse(responseText, actionType.speaker);
                    if (actionType.useFullHistory) {
                         gameState.history.push({ role: "user", parts: [{ text: prompt }] });
                         gameState.history.push({ role: "model", parts: [{ text: responseText }] });
                    }
                } else {
                    addStoryParagraph("The air grows thick with silence... something went wrong.", "narrator");
                }
            } catch (error) {
                console.error("Error with Gemini API:", error);
                addStoryParagraph(`A thick fog rolls into your mind, obscuring your thoughts. (Error: ${error.message})`, "narrator");
            } finally {
                showLoader(false);
            }
        }

        function getActionType(action) {
            if (action.startsWith("Analyze")) return { speaker: 'analysis', loaderText: 'Analyzing...', useFullHistory: false };
            if (action.startsWith("Press")) return { speaker: 'player', loaderText: 'Applying pressure...', useFullHistory: true };
            if (action === "I need a hint") return { speaker: 'hint', loaderText: 'Finding a thread...', useFullHistory: false };
            if (action === "Reconstruct the scene") return { speaker: 'reconstruction', loaderText: 'Reconstructing scene...', useFullHistory: false };
            return { speaker: 'player', loaderText: 'Thinking...', useFullHistory: true };
        }

        function parseAndApplyResponse(text, speaker) {
            const narrative = (text.match(/NARRATIVE:([\s\S]*?)(?=INVENTORY_ADD:|CLUES_DISCOVERED:|CHARACTER_UPDATE:|$)/) || [])[1]?.trim() || text;

            if (speaker === 'hint' || speaker === 'reconstruction' || speaker === 'analysis') {
                 addStoryParagraph(narrative, speaker);
                 return;
            }

            const newItems = JSON.parse((text.match(/INVENTORY_ADD:(\[.*?\])/) || [])[1] || '[]');
            const newClues = JSON.parse((text.match(/CLUES_DISCOVERED:(\[.*?\])/) || [])[1] || '[]');
            const charUpdates = JSON.parse((text.match(/CHARACTER_UPDATE:(\[.*?\])/) || [])[1] || '[]');

            addStoryParagraph(narrative, "narrator");

            newItems.forEach(item => { if (!gameState.inventory.includes(item)) gameState.inventory.push(item); });
            newClues.forEach(clue => { if (!gameState.known_clues.includes(clue)) gameState.known_clues.push(clue); });
            charUpdates.forEach(update => {
                if (gameState.characters[update.name]) {
                    gameState.characters[update.name].status = update.status;
                }
            });

            const spokenToMatch = narrative.match(/^(\w+\s?\w*): "/);
            gameState.lastSpokenTo = spokenToMatch ? spokenToMatch[1] : null;

            if(gameState.lastSpokenTo && gameState.characters[gameState.lastSpokenTo]){
                 gameState.characters[gameState.lastSpokenTo].dialogue.push(narrative);
            }

            updateUI();
        }

        function createPrompt(playerAction) {
            if (playerAction === "I need a hint") {
                return `You are a game master. The player is stuck. Provide a subtle, in-character hint based on the game state. Do not give away the solution. Frame it as the detective's inner monologue. GAME STATE: Known Clues: ${JSON.stringify(gameState.known_clues)}, Inventory: ${JSON.stringify(gameState.inventory)}, Characters: ${JSON.stringify(gameState.characters)}. FORMAT: NARRATIVE: [Your hint]`;
            }
            if (playerAction === "Reconstruct the scene") {
                 return `You are a forensic analyst AI. Based ONLY on the following clues, construct a plausible, speculative narrative of how the crime might have occurred. It's okay to mention contradictions or missing pieces. Present it as a professional summary. KNOWN CLUES: ${JSON.stringify(gameState.known_clues)}. FORMAT: NARRATIVE: [Your reconstruction]`;
            }
            if (playerAction.startsWith("Analyze")) {
                const item = playerAction.replace("Analyze ", "");
                return `You are a forensic expert. The detective wants you to analyze an item. Provide a detailed analysis based on the item and the known clues. What could it mean? What are the possibilities? ITEM: "${item}". KNOWN CLUES: ${JSON.stringify(gameState.known_clues)}. FORMAT: NARRATIVE: [Your analysis]`;
            }
             if (playerAction.startsWith("Press")) {
                const characterName = playerAction.replace("Press ", "").replace(" for details", "");
                const character = gameState.characters[characterName];
                return `You are a master storyteller game master. The player is pressing a suspect for more information. The character should react to the pressure—either by revealing something new, getting defensive, or telling a lie. PLAYER'S ACTION: "${playerAction}". CHARACTER TO PRESS: ${characterName}. THEIR CURRENT STATUS: ${character.status}. PREVIOUS DIALOGUE WITH THEM: ${JSON.stringify(character.dialogue)}. KNOWN CLUES: ${JSON.stringify(gameState.known_clues)}. YOUR TASK: Write the next part of the conversation. Update game state. FORMAT: NARRATIVE: [dialogue/outcome] INVENTORY_ADD: [] CLUES_DISCOVERED: [] CHARACTER_UPDATE: [{"name": "${characterName}", "status": "[new status]"}]`;
            }

             return `You are a master storyteller game master. Your tone is noir and mysterious. PLAYER'S ACTION: "${playerAction}". GAME STATE: Characters: ${JSON.stringify(gameState.characters)}, Inventory: ${JSON.stringify(gameState.inventory)}, Known Clues: ${JSON.stringify(gameState.known_clues)}. YOUR TASK: Describe the outcome. Update game state. If the player makes an accusation, determine if they have enough evidence and narrate the dramatic conclusion (success or failure). FORMAT: NARRATIVE: [story text] INVENTORY_ADD: [] CLUES_DISCOVERED: [] CHARACTER_UPDATE: []`;
        }

        function showLoader(isLoading, text = "Thinking...") {
            loaderText.textContent = text;
            loader.classList.toggle('hidden', !isLoading);
            loader.classList.toggle('flex', isLoading);
            document.getElementById('input-area').classList.toggle('hidden', isLoading);
        }

        function addEventListeners() {
            quickActionsContainer.onclick = function(e) {
                if (e.target.matches('.quick-action-btn')) {
                    const action = e.target.dataset.action;
                    playerInput.value = action;
                    handlePlayerAction(action);
                }
            };

            inventoryList.onclick = function(e) {
                if (e.target.matches('.analyze-btn')) {
                    const item = e.target.dataset.item;
                    handlePlayerAction(`Analyze ${item}`);
                }
            }
        }

        inputForm.addEventListener('submit', (e) => {
            e.preventDefault();
            handlePlayerAction(playerInput.value);
        });

        reconstructButton.addEventListener('click', () => {
             handlePlayerAction("Reconstruct the scene");
        });

        window.onload = startGame;

    </script>
</body>
</html>


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 251)